# Load the dataset and scikit-learn modules

In [32]:
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as p1
from scipy.stats import spearmanr as p2
from scipy.stats import pointbiserialr as p3

import pandas as pd
import numpy as np
from sklearn.svm import SVR
import nltk

#raw_data = pd.read_csv("combined_dataset_beta.csv")
raw_data = pd.read_csv("combined_dataset_charlie.csv")
# raw_data = raw_data.sample(frac=0.01).reset_index(drop=True)
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

raw_data = raw_data.fillna(value = 0)
raw_data = raw_data.sample(frac = 1.0, random_state = 1)


# Split the data into test and train

In [33]:
train = raw_data.sample(frac = 0.8, random_state = 1)
test = raw_data.loc[~raw_data.index.isin(train.index)]

# Selecting features to use for the regression model

In [34]:
data_columns = [
 'yules_i_for_abs',
 'avg_word_len_abs',
 'avg_sen_len_abs',
 'freq_of_words_great_sent_len_abs',
 'yules_i_for_blg',
 'avg_word_len_blg',
 'avg_sen_len_blg',
 'freq_of_words_great_sent_len_blg',
 'bigrams_abs_blg',
 'trigrams_abs_blg',
#  'mendeley_readers',
#  'citeulikereaders',
#  'connoteareaders',
#  'blog_users',
#  'blogs_posts_count',
#  'news_unique_users',
#  'total_posts_count',
#  'wiki_posts_count',
#  'facebook_users',
#  'facebook_posts',
#  'twitter_users',
#  'twitter_posts',
#  'citation_page',
#  'other_articles',
#  'mean',
#  'rank',
#  'perc',
#  'scored_higher_than',
#  'sample_size',
 ]

# Predicting the similairty score

In [35]:
train_data_array = train.as_matrix(columns = data_columns)
train_target_array = train['similarity_score'].values
test_data_array = test.as_matrix(columns = data_columns)
test_target_array = test['similarity_score'].values

# Regression on the Dataset : {KNN, DecisionTree and Random Forest}

In [36]:
from sklearn import linear_model
from sklearn.metrics import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering

clf = linear_model.Lasso()
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
# svr_poly = SVR(kernel='poly', C=1e3, degree=2)

# print("Linear")
# clf.fit(train_data_array, train_target_array)
# y = clf.predict(test_data_array)
# print(mean_squared_error(test_data_array, y))
# print(r2_score(test_target_array, y), "\n")

print("KNN")
n =  KNeighborsRegressor(n_neighbors = 17)
n.fit(train_data_array, train_target_array)
y = n.predict(test_data_array)
print(mean_squared_error(test_target_array, y))
print(r2_score(test_target_array, y), "\n")

print("DecisionTree")
n =  DecisionTreeRegressor(criterion='mse')
n.fit(train_data_array, train_target_array)
y = n.predict(test_data_array)
print(mean_squared_error(test_target_array, y))
print(r2_score(test_target_array, y), "\n")

print("Random Forest")
n =  RandomForestRegressor()
n.fit(train_data_array, train_target_array)
y = n.predict(test_data_array)
print(mean_squared_error(test_target_array, y))
print(r2_score(test_target_array, y), "\n")

KNN
0.00436179832646
0.405058310859 

DecisionTree
0.0022091176009
0.698680209717 

Random Forest
0.00143906282422
0.803714339055 



# KNN Clustering on entire Dataset

In [37]:
# print("KNN Cluster")
# n =  KMeans(n_clusters=5)
# n.fit(train_data_array)
# y = n.predict(test_data_array)
# labels = n.labels_
# centroids = n.cluster_centers_
# print(centroids)
# print(mean_squared_error(test_target_array, y))
# print(r2_score(test_target_array, y), "\n")
print(p1(test_target_array, y))
print(p2(test_target_array, y))
print(p3(test_target_array, y))

(0.89655458698580481, 0.0)
SpearmanrResult(correlation=0.91940630040635218, pvalue=0.0)
PointbiserialrResult(correlation=0.89655458698580481, pvalue=0.0)


# SVM rbf on the entire Dataset

In [ ]:
print("SVM")
n =  SVR(kernel='rbf')
n.fit(train_data_array, train_target_array)
y = n.predict(test_data_array)
print(mean_squared_error(test_target_array, y))
print(r2_score(test_target_array, y), "\n")

svr_rbf.fit(train_data_array, train_target_array).predict(train_data_array)
svr_rbf.score(test_data_array, test_target_array)

# y_rbf = svr_rbf.fit(train_data_array, train_target_array).predict(train_data_array)
# y_poly = svr_poly.fit(train_data_array, train_target_array).predict(train_data_array)
# svr_poly.score(test_data_array, test_target_array)
# y_poly.score(test_data_array, test_target_array)

# K Fold Cross Validation

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=10, random_state=None, shuffle=False)
validation_score=0
for train_index, test_index in kf.split(train_data_array):
    train_data_array, X_test = train_data_array[train_index], train_data_array[test_index]
    y_train, y_test = train_class_array[train_index], train_class_array[test_index]
    clf.fit(X_train, y_train)
    validation_pred=clf.predict(X_test)
    validation_score+=metrics.accuracy_score(y_test,validation_pred)